<a href="https://colab.research.google.com/github/sandeshar/App-autostarter/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:

Transforms=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5],[0.5])])
trainset=datasets.MNIST('',train=True, download=True, transform=Transforms)
testset=datasets.MNIST('',train=False, transform=Transforms)

train_loader=torch.utils.data.DataLoader(trainset,batch_size=64, shuffle=True)
test_loader=torch.utils.data.DataLoader(testset,batch_size=64, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw



In [ ]:
class NN(nn.Module):
 def __init__(self):
  super(NN,self).__init__()
  self.conv1=nn.Conv2d(1,32,3)
  self.conv2=nn.Conv2d(32,64,3)
  self.conv3=nn.Conv2d(64,128,3)
  
  x=torch.randn(28,28).view(-1,1,28,28)
  self._to_linear=None
  self.convs(x)

  self.f1=nn.Linear(self._to_linear,64)
  self.f2=nn.Linear(64,10)

 def convs(self,x):
  x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
  x=F.max_pool2d(F.relu(self.conv2(x)),(2,2))
  x=F.max_pool2d(F.relu(self.conv3(x)),(2,2))

  if self._to_linear is None:
   self._to_linear=x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
  return x

 def forward(self,x):
  x=self.convs(x)
  x=x.view(-1,self._to_linear)
  x=F.relu(self.f1(x))
  x=F.softmax(self.f2(x),dim=1)
  return x

In [ ]:
model=NN()
loss_f=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

for epoch in range(3):
  for data in train_loader:
    x,y= data
    output=model(x)
    optimizer.zero_grad()
    loss=loss_f(output,y)
    loss.backward()
    optimizer.step()
  print(str(epoch)+" epoch and loss = "+str(loss))
print("done")

0 epoch and loss = tensor(1.4707, grad_fn=<NllLossBackward0>)
1 epoch and loss = tensor(1.4699, grad_fn=<NllLossBackward0>)
2 epoch and loss = tensor(1.4943, grad_fn=<NllLossBackward0>)
done


In [ ]:
def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)

accuracy = testAccuracy()
print(accuracy)

97.7
